In [1]:
import pandas as pd
import os
mingw_path = 'C:\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'

os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

In [181]:
import xgboost as xg
import datetime
import sklearn.preprocessing as pp

In [56]:
train = pd.read_csv('CrossSellTarget\\Train.csv',)

In [7]:
test = pd.read_csv('CrossSellTarget\\Test.csv')

C:\Users\Arkajit\Anaconda2\envs\TFENV\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,26,28,29,30,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [57]:
y_train = train['RESPONDERS']

In [58]:
train.dropna(axis=1,how='all' ,inplace=True)

In [8]:
test.dropna(axis=1,how='all',inplace=True)

In [14]:
len(train.columns)

323

In [15]:
len(test.columns)

323

In [16]:
train.columns

Index([u'CUSTOMER_ID', u'ACT_TYPE', u'GENDER', u'LEGAL_ENTITY',
       u'OCCUP_ALL_NEW', u'AGE', u'N_CASA_MAX_BALANCE_MTD',
       u'N_CASA_MIN_BALANCE_MTD', u'BL_TAG', u'TWL_TAG',
       ...
       u'EOP_MON_03', u'CREDIT_ACT', u'DEBIT_ACT', u'CEL_TAG', u'EL_TAG',
       u'AGL_TAG', u'CVL_TAG', u'AL_TAG', u'HL_EMI', u'SCRUB_EMI'],
      dtype='object', length=323)

In [59]:
for i in train.columns:
    if(i not in test.columns):
        print i

BBG_TAG
BBG_CLOSED
RESPONDERS


In [60]:
train.drop(['BBG_TAG','BBG_CLOSED','RESPONDERS','ZIP_CODE_FINAL'],axis=1 , inplace=True)
test.drop(['ZIP_CODE_FINAL'],axis=1,inplace= True)

In [62]:
for i in train.columns:
    if len(train[i].astype('category').cat.categories.get_values()) <= 3:
        train[i] = train[i].astype('category').cat.codes

In [67]:
for i in test.columns:
    if len(test[i].astype('category').cat.categories.get_values()) <= 3:
        test[i] = test[i].astype('category').cat.codes

In [68]:
dict_train = {}
dict_test = {}

In [169]:
train['MATURITY_GL_YR'] = train['MATURITY_GL'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').year if str(x)  != str(pd.np.nan) else 0 )
train['MATURITY_GL_MN'] = train['MATURITY_GL'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').month if str(x)  != str(pd.np.nan) else 0 )


train['MATURITY_LAP_YR'] = train['MATURITY_LAP'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').year if str(x)  != str(pd.np.nan) else 0 )
train['MATURITY_LAP_MN'] = train['MATURITY_LAP'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').month if str(x)  != str(pd.np.nan) else 0 )


train['MATURITY_LAS_YR'] = train['MATURITY_LAS'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').year if str(x)  != str(pd.np.nan) else 0 )
train['MATURITY_LAS_MN'] = train['MATURITY_LAS'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').month if str(x)  != str(pd.np.nan) else 0 )


train['CLOSED_DATE_YR'] = train['CLOSED_DATE'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').year if str(x)  != str(pd.np.nan) else 0 )
train['CLOSED_DATE_MN'] = train['CLOSED_DATE'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').month if str(x)  != str(pd.np.nan) else 0 )

In [174]:
test['MATURITY_GL_YR'] = test['MATURITY_GL'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').year if str(x)  != str(pd.np.nan) else 0 )
test['MATURITY_GL_MN'] = test['MATURITY_GL'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').month if str(x)  != str(pd.np.nan) else 0 )


test['MATURITY_LAP_YR'] = test['MATURITY_LAP'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').year if str(x)  != str(pd.np.nan) else 0 )
test['MATURITY_LAP_MN'] = test['MATURITY_LAP'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').month if str(x)  != str(pd.np.nan) else 0 )


test['MATURITY_LAS_YR'] = test['MATURITY_LAS'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').year if str(x)  != str(pd.np.nan) else 0 )
test['MATURITY_LAS_MN'] = test['MATURITY_LAS'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').month if str(x)  != str(pd.np.nan) else 0 )


test['CLOSED_DATE_YR'] = test['CLOSED_DATE'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').year if str(x)  != str(pd.np.nan) else 0 )
test['CLOSED_DATE_MN'] = test['CLOSED_DATE'].apply(lambda x: datetime.datetime.strptime(str(x),'%d%b%Y').month if str(x)  != str(pd.np.nan) else 0 )

In [176]:
train.drop(['MATURITY_LAS','CLOSED_DATE','MATURITY_LAP','MATURITY_GL'],axis=1,inplace=True)

In [177]:
test.drop(['MATURITY_LAS','CLOSED_DATE','MATURITY_LAP','MATURITY_GL'],axis=1,inplace=True)

In [206]:
cat_cols=[#'LEGAL_ENTITY','DESIGNATION_FINAL',
          'PA_PQ_TAG','NEFT_CC_CATEGORY','NEFT_DC_CATEGORY','TPT_DC_CATEGORY_MON_01',
'TPT_CC_CATEGORY_MON_01','IMPS_CC_CATEGORY_MON_01'
    #,'CLOSED_DATE_YR','CLOSED_DATE_MN','MATURITY_LAP_YR','MATURITY_LAP_MN',
#'MATURITY_LAS_YR','MATURITY_LAS_MN','MATURITY_GL_YR','MATURITY_GL_MN'
]

In [197]:
for i in train.columns:
    if (len(train[i].astype('category').cat.categories.get_values()) > 3) & (str(train[i].dtype) == 'object'):
        print i

PA_PQ_TAG
NEFT_CC_CATEGORY
NEFT_DC_CATEGORY
TPT_DC_CATEGORY_MON_01
TPT_CC_CATEGORY_MON_01
IMPS_CC_CATEGORY_MON_01


In [209]:
for col in cat_cols:
    print col
    train[col] = train[col].apply(lambda x: x if str(x)  != str(pd.np.nan) else 0 )
    test[col] = test[col].apply(lambda x: x if str(x)  != str(pd.np.nan) else 0 )
    data= test[col].append(train[col])
    le = pp.LabelEncoder().fit(data)
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])
    

PA_PQ_TAG
NEFT_CC_CATEGORY
NEFT_DC_CATEGORY
TPT_DC_CATEGORY_MON_01
TPT_CC_CATEGORY_MON_01
IMPS_CC_CATEGORY_MON_01


In [223]:
train.drop(['OCCUP_ALL_NEW'],axis=1,inplace = True)
test.drop(['OCCUP_ALL_NEW'],axis=1,inplace = True)

In [230]:
le = pp.LabelEncoder().fit(y_train)
y_train = le.transform(y_train)
y_train

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [244]:
x_train = train.as_matrix()
x_test = test.as_matrix()

In [238]:
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

C:\Users\Arkajit\Anaconda2\envs\TFENV\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Arkajit\Anaconda2\envs\TFENV\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [242]:
train.index = train['CUSTOMER_ID']
train.drop(['CUSTOMER_ID'],axis=1,inplace=True)

In [243]:
test.index = test['CUSTOMER_ID']
test.drop(['CUSTOMER_ID'],axis=1,inplace=True)